In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 

In [ ]:
userInput

In [10]:
def movies(userInput):
    inputMovies = pd.DataFrame(userInput)
    df = pd.read_csv('movies.csv')
    users = pd.read_csv('ratings.csv')
    del users['timestamp']
    lista= []
    for i in range(len(df)):
        lista.append(df.iloc[i,1].split('(')[0].strip()) #clean titles from our df with movies info 
    df['title'] = lista
    df2 = pd.merge(left = inputMovies, right = df, how = 'inner', on = 'title')
    del df2['title']
    del df2['genres']
    filtrado = users[users['movieId'].isin(df2['movieId'].values)]
    
    #filter by users that saw the same movies as our user
    from collections import Counter
    dic = dict(Counter(filtrado['userId']))
    lista2 = []
    for i in dic.items(): 
        if i[1] == 5:
            lista2.append(i[0])

    user100 = filtrado[filtrado['userId'].isin(lista2[0:100])] #choose 100 users
    user100 = user100.reset_index(drop = True)

    columns = ['userId']

    for i in df2['movieId'].values:
        columns.append(i)

    df_agrupado = pd.DataFrame(columns=columns)
    df_agrupado #create new df with users, movies to make correlations

    for i in range(len(lista2[0:100])): #fill the df with user ratings and movie ids in columns
        registro = []
        registro.append(lista2[0:100][i])
        for j in df_agrupado.columns[1:]:
            registro.append(float(user100[(user100["userId"] == lista2[0:100][i]) & (user100["movieId"] == j)]["rating"].values))
        df_agrupado.loc[i] = registro

    from scipy import stats

    corr= [] #calculate correlations between our user ratings and similar customer ratings 
    for i in range(len(df_agrupado)):
        corr.append(stats.pearsonr(df_agrupado.iloc[i,1:],df2['rating'])[0])

    df_agrupado['corr'] = corr
    #create a new df with the same users but the movies our user haven't seen
    df_agrupado2 = pd.merge(left = df_agrupado, right = users[(users['userId'].isin(lista2[0:100])) & (~users['movieId'].isin(df2['movieId'].values))], how = 'inner', on = ['userId'])

    #multipy the correlations (similarity index and ratings of each movie)
    notas = []
    for i in range(len(df_agrupado2)):
        notas.append(df_agrupado2.iloc[i,6] * df_agrupado2.iloc[i,8])

    df_agrupado2['notas'] = notas

    dffinal = df_agrupado2.groupby('movieId').sum().reset_index().sort_values('movieId') #sum ratings of each movie
    dffinal.drop( df2['movieId'].values , axis=1, inplace= True)

    ratingfinal = [] #calculate final rating dividing the movie rating and the correlation of the user with our user 
    for i in range(len(dffinal)):
        if dffinal.iloc[i,2] == 0:
            ratingfinal.append(0)
        else:
            ratingfinal.append(dffinal.iloc[i,4] / dffinal.iloc[i,2])

    dffinal['ratingfinal'] = ratingfinal

    dffinal = pd.merge(left = dffinal, right = df, how = 'inner', on = 'movieId') #add the initial df with movie titles

    dffinal.sort_values('ratingfinal', ascending = False)

    return dffinal.sort_values('ratingfinal', ascending = False).head(10)['title'].values #return 10 movie titles 

In [11]:
movies(userInput)

array(['R.I.P.D.', 'Under the Rainbow', 'Our Family Wedding',
       'He Was a Quiet Man', 'Did You Hear About the Morgans?',
       'Killing Them Softly', 'Children of Huang Shi, The', 'Everest',
       'Bell, Book and Candle', 'Auntie Mame'], dtype=object)